In [2]:

import gensim
import pandas as pd
import scipy as sp
print("done1")

D:\Users\yelfs\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
done1


In [3]:
# Exports data from corpus
sentences, cur_sent = list(), list()
with open('./wackypedia_en1.words10.20Mwords') as f:
 for line in f:
    line = line.strip()
    if line == '</s>':
        sentences.append(cur_sent)
        cur_sent = list()
    elif line != '<s>' and not line.startswith('<text') and not line.startswith('</text'):
        cur_sent.append(line.split('\t')[0])
print("done2")

done2


In [4]:
# Builds the models
model_big_ten = gensim.models.Word2Vec(sentences, min_count=5, window=1, vector_size=10)
model_small_ten = gensim.models.Word2Vec(sentences, min_count=5, window=10, vector_size=10)

model_big_five = gensim.models.Word2Vec(sentences, min_count=5, window=1, vector_size=500)
model_small_five = gensim.models.Word2Vec(sentences, min_count=5, window=10, vector_size=500)

models = [model_big_ten, model_small_ten, model_big_five, model_small_five]
print("done3")

done3


In [5]:

# Makes the similarity lists
words = pd.read_csv('./SimLex-999/SimLex-999.txt', delimiter='\t')
pairs = list(zip(words['word1'],words['word2']))
print("done4")

done4


In [6]:
# Calculate similarity and write to files
results = [[], [], [], []]
pos = words['POS']
models_data = ["size10_window1", "size10_window10", "size500_window1", "size500_window10"]
for j in range(len(models)):
    f = open(models_data[j],"w+")
    for i in range(len(pairs)):
        try:
            similarity = models[j].wv.similarity(pairs[i][0], pairs[i][1])
        except KeyError:
            similarity = 0
        finally:
            results[j].append(similarity)
            f.write(pairs[i][0] + "\t" + pairs[i][1] + "\t" + pos[i] + "\t" + str(similarity) + "\n")
print("done5")

done5


In [7]:
# Calculate overall correlation
cor_results = []
golden_sim = words['SimLex999']
for result in results:
    sim_res = sp.stats.spearmanr(result, golden_sim)
    cor_results.append(sim_res)
print(cor_results)

[SpearmanrResult(correlation=0.1267090606417617, pvalue=5.9176296353660535e-05), SpearmanrResult(correlation=0.2020214109355799, pvalue=1.164863083951351e-10), SpearmanrResult(correlation=0.2141625662511345, pvalue=7.915633078730475e-12), SpearmanrResult(correlation=0.23801322524877044, pvalue=2.4732725873936907e-14)]


In [11]:
# Get noun correleation
pos = words['POS']
n_results = [[], [], [], []]

for j in range(len(models)):
    for i in range(len(pairs)):
        if pos[i] == 'N':
            try:
                similarity = models[j].wv.similarity(pairs[i][0], pairs[i][1])
            except KeyError:
                similarity = 0
            finally:
                n_results[j].append(similarity)

n_golden = []
for i in range(len(pairs)):
    if pos[i] == 'N':
        n_golden.append(words['SimLex999'][i])

n_cor_results = []

for result in n_results:
    sim_res, pval = sp.stats.spearmanr(result, n_golden)
    n_cor_results.append(sim_res)
print(n_cor_results)


[0.12232063598153842, 0.22873673735132238, 0.20493154689075563, 0.23527904302198654]


In [12]:
# Get adjective correlation
pos = words['POS']
n_results = [[], [], [], []]

for j in range(len(models)):
    for i in range(len(pairs)):
        if pos[i] == 'A':
            try:
                similarity = models[j].wv.similarity(pairs[i][0], pairs[i][1])
            except KeyError:
                similarity = 0
            finally:
                n_results[j].append(similarity)

n_golden = []
for i in range(len(pairs)):
    if pos[i] == 'A':
        n_golden.append(words['SimLex999'][i])

n_cor_results = []

for result in n_results:
    sim_res, pval = sp.stats.spearmanr(result, n_golden)
    n_cor_results.append(sim_res)
print(n_cor_results)


[0.21273823069807435, 0.22319436475783655, 0.3089627459857727, 0.27468084275080756]


In [8]:
# get verbs correlation
pos = words['POS']
n_results = [[], [], [], []]

for j in range(len(models)):
    for i in range(len(pairs)):
        if pos[i] == 'V':
            try:
                similarity = models[j].wv.similarity(pairs[i][0], pairs[i][1])
            except KeyError:
                similarity = 0
            finally:
                n_results[j].append(similarity)

n_golden = []
for i in range(len(pairs)):
    if pos[i] == 'V':
        n_golden.append(words['SimLex999'][i])

n_cor_results = []

for result in n_results:
    sim_res, pval = sp.stats.spearmanr(result, n_golden)
    n_cor_results.append(sim_res)
print(n_cor_results)


[0.08472524350619494, 0.10564962010490918, 0.22658587000780966, 0.21715141881979055]


In [17]:
# Print analogy by model

for index, model in enumerate(models):
    results = []
    print("--------------------------------")
    print(models_data[index] + " RESULTS")
    print("--------------------------------")
    results.append(model.wv.most_similar(positive=['dog', 'wife'], negative=['cat']))
    print(results)


--------------------------------
size10_window1 RESULTS
--------------------------------
[[('widow', 0.9832034707069397), ('brother', 0.9827520251274109), ('son', 0.9822415113449097), ('cousin', 0.9805023074150085), ('nephew', 0.978701651096344), ('daughter', 0.97786545753479), ('grandmother', 0.9726814031600952), ('uncle', 0.9720106720924377), ('grandfather', 0.9687551856040955), ('niece', 0.9683469533920288)]]
--------------------------------
size10_window10 RESULTS
--------------------------------
[[('married', 0.9673024415969849), ('sister', 0.960009753704071), ('Troy', 0.9545236825942993), ('Catherine', 0.9468388557434082), ('Titus', 0.9466161131858826), ('brother', 0.9464559555053711), ('Anne', 0.9455770254135132), ('murdered', 0.9450943470001221), ('daughter', 0.9438880085945129), ('Alfonso', 0.9417060017585754)]]
--------------------------------
size500_window1 RESULTS
--------------------------------
[[('widow', 0.6843394637107849), ('girlfriend', 0.6738806366920471), ('niece'

In [18]:
# Print analogy by model
for index, model in enumerate(models):
    results = []
    print("--------------------------------")
    print(models_data[index] + " RESULTS")
    print("--------------------------------")
    results.append(model.wv.most_similar(positive=['light', 'right'], negative=['dark']))
    print(results)

--------------------------------
size10_window1 RESULTS
--------------------------------
[[('rise', 0.9800962209701538), ('forward', 0.9667182564735413), ('up', 0.9434640407562256), ('refuge', 0.9377111196517944), ('westward', 0.9349880814552307), ('ban', 0.933565080165863), ('out', 0.9298392534255981), ('back', 0.9286207556724548), ('possession', 0.9285981059074402), ('suit', 0.9278940558433533)]]
--------------------------------
size10_window10 RESULTS
--------------------------------
[[('able', 0.9443411231040955), ('force', 0.9265884160995483), ('removed', 0.8982120752334595), ('passing', 0.8868582248687744), ('ready', 0.8801742792129517), ('allowed', 0.8768723011016846), ('charge', 0.874687910079956), ('pressed', 0.8739463686943054), ('order', 0.8736936450004578), ('required', 0.8671567440032959)]]
--------------------------------
size500_window1 RESULTS
--------------------------------
[[('freedom', 0.47313764691352844), ('advance', 0.46372777223587036), ('force', 0.4615896940231

In [19]:
# Print analogy by model
for index, model in enumerate(models):
    results = []
    print("--------------------------------")
    print(models_data[index] + " RESULTS")
    print("--------------------------------")
    results.append(model.wv.most_similar(positive=['new', 'man'], negative=['old']))
    print(results)

--------------------------------
size10_window1 RESULTS
--------------------------------
[[('difficulty', 0.9592686891555786), ('patient', 0.9415267109870911), ('possibility', 0.9373559951782227), ('truly', 0.9363977909088135), ('increasingly', 0.9333330988883972), ('seemingly', 0.9332449436187744), ('crime', 0.930452823638916), ('incapable', 0.9296925067901611), ('situation', 0.9232394099235535), ('best', 0.9209893941879272)]]
--------------------------------
size10_window10 RESULTS
--------------------------------
[[('copyright', 0.9511670470237732), ('motivation', 0.9128372073173523), ('examination', 0.9108969569206238), ('appeal', 0.9087449312210083), ('action', 0.9044215679168701), ('inquiry', 0.8968518376350403), ('decisions', 0.8849015235900879), ('procedure', 0.8809605836868286), ('defendant', 0.8809313178062439), ('credibility', 0.877590000629425)]]
--------------------------------
size500_window1 RESULTS
--------------------------------
[[('person', 0.47325262427330017), ('wo

In [22]:
# Print analogy by model
for index, model in enumerate(models):
    results = []
    print("--------------------------------")
    print(models_data[index] + " RESULTS")
    print("--------------------------------")
    results.append(model.wv.most_similar(positive=['white', 'big'], negative=['black']))
    print(results)

--------------------------------
size10_window1 RESULTS
--------------------------------
[[('prosperous', 0.9776113629341125), ('consuming', 0.9714850783348083), ('productive', 0.962975025177002), ('breeding', 0.9611977338790894), ('regular', 0.9564952254295349), ('distant', 0.9551976919174194), ('lean', 0.9528768062591553), ('active', 0.952736496925354), ('respectable', 0.9509806632995605), ('favourable', 0.9458457827568054)]]
--------------------------------
size10_window10 RESULTS
--------------------------------
[[('eight', 0.9217930436134338), ('six', 0.9133078455924988), ('four', 0.9059357047080994), ('five', 0.8998093605041504), ('top', 0.8926584720611572), ('ten', 0.8908166289329529), ('golf', 0.8779656887054443), ('regular', 0.8765390515327454), ('Gator', 0.8669626116752625), ('tennis', 0.8603619337081909)]]
--------------------------------
size500_window1 RESULTS
--------------------------------
[[('bright', 0.6023468375205994), ('busy', 0.581092119216919), ('fine', 0.5800844

In [21]:
# Print analogy by model
for index, model in enumerate(models):
    results = []
    print("--------------------------------")
    print(models_data[index] + " RESULTS")
    print("--------------------------------")
    results.append(model.wv.most_similar(positive=['white', 'up'], negative=['black']))
    print(results)

--------------------------------
size10_window1 RESULTS
--------------------------------
[[('down', 0.9800690412521362), ('away', 0.977653980255127), ('storm', 0.9719418883323669), ('off', 0.9693686962127686), ('out', 0.9658371806144714), ('apart', 0.9584853053092957), ('steadily', 0.9568665623664856), ('tickets', 0.9446605443954468), ('back', 0.9410886168479919), ('rapidly', 0.9394614696502686)]]
--------------------------------
size10_window10 RESULTS
--------------------------------
[[('off', 0.8984220623970032), ('away', 0.8631198406219482), ('back', 0.8602479696273804), ('household', 0.8543658256530762), ('down', 0.8539766669273376), ('out', 0.8521571159362793), ('caught', 0.8473911285400391), ('remaining', 0.8438048958778381), ('rushing', 0.8331202268600464), ('traveling', 0.8278323411941528)]]
--------------------------------
size500_window1 RESULTS
--------------------------------
[[('off', 0.5624390244483948), ('down', 0.5223851203918457), ('forth', 0.5061142444610596), ('asid